<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/ER_gemini3_thinking_level_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

API Parameters:

thinking_level: Set to "high" (for Deep Think) or "low" (for faster, cheaper responses).

include_thoughts: Set to True to get the model's intermediate thinking steps.

In [1]:
# 1. IMPORTS & API SETUP
import google.genai as genai
from google.genai import types
import os

# --- Secure API Client Initialization using Userdata/Environment Variables ---
GEMINI_API_KEY = None
try:
    # Attempt to load from Colab secrets ('GEMINI')
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI')
except (ImportError, KeyError):
    # Fallback to standard environment variable
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

REQUESTED_MODEL_ID = 'gemini-3-pro-preview'
client = None

if GEMINI_API_KEY:
    try:
        client = genai.Client(api_key=GEMINI_API_KEY)
        print(f"✅ Gemini client configured for **{REQUESTED_MODEL_ID}**.")
    except Exception as e:
        print(f"❌ Client initialization failed: {e}")
else:
    print("❌ API Key not found. Please ensure your key is set up.")


# --- 2. AGENT CONFIGURATIONS ---

# 🎯 Agent 1: Triage Nurse Agent (Low Thinking)
def get_low_think_config():
    """Returns config for fast, low-cost operations (e.g., triage classification, data extraction)."""
    return types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            # Minimizes internal deliberation for fast, protocol-based checks.
            thinking_level="low"
        )
    )

# 🎯 Agent 2: Consulting Physician Agent (High Thinking / Deep Think Mode)
def get_high_think_config():
    """Returns config for rigorous, differential diagnosis and complex decision-making."""
    return types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            # Maximizes internal deliberation for complex, high-stakes medical diagnosis.
            thinking_level="high",
            # Request internal thoughts for auditability of the diagnostic chain.
            include_thoughts=True
        )
    )

# --- 3. MULTI-AGENT ORCHESTRATOR SIMULATION ---

def ed_orchestrator(system_prompt: str):
    """Simulates the ED supervisor delegating tasks to Triage and Consulting Agents."""

    if not client:
        print("\nOrchestrator cannot run without a configured Gemini client.")
        return

    print("\n=======================================================")
    print(f"ED Supervisor receives new task: {system_prompt}\n")

    # Task 1: Triage Triage & Data Extraction (Delegated to Low-Think Agent)
    # Goal: Quickly assign an Emergency Severity Index (ESI) and extract key vitals.
    low_think_prompt = (
        "Based on the following triage notes, assign an ESI level (1-5) and extract the patient's heart rate: "
        "55yo Male presenting with fever, cough, and mild shortness of breath. HR: 110, BP: 140/90, SpO2: 94% on room air. "
        "Output ONLY a single JSON object with keys 'esi_level' and 'heart_rate'."
    )

    print("--- 🟢 DELEGATING TO LOW-THINK AGENT (Triage/Fast Data) ---")
    response_low = client.models.generate_content(
        model=REQUESTED_MODEL_ID,
        contents=low_think_prompt,
        config=get_low_think_config()
    )
    print(f"Agent 1 Result (LOW THINK): {response_low.text.strip()}")
    print("-" * 50)

    # Task 2: Differential Diagnosis (Delegated to High-Think Agent)
    # Goal: Analyze ambiguous symptoms and lab results to create a differential diagnosis.
    high_think_prompt = (
        "A 72yo patient presents with acute, severe abdominal pain that radiates to the back, "
        "accompanied by hypotension (BP 90/60) and a pulsatile mass on exam. The patient is refusing consent for imaging. "
        "Given the high likelihood of a ruptured AAA, formulate the optimal 3-step immediate management plan, "
        "prioritizing stabilization and ethical consent strategy. Detail the reasoning for the top two differential diagnoses."
    )

    print("--- 🔴 DELEGATING TO HIGH-THINK AGENT (Diagnosis/Deep Think Mode) ---")
    response_high = client.models.generate_content(
        model=REQUESTED_MODEL_ID,
        contents=high_think_prompt,
        config=get_high_think_config()
    )

    # Extract Thought Summary for Auditability
    thought_summary = next((part.text for part in response_high.candidates[0].content.parts if hasattr(part, 'thought') and part.thought), None)

    print(f"Agent 2 Result (HIGH THINK):\n{response_high.text.strip()}")

    if thought_summary:
        print("\n*** DEEP THINK INTERNAL REASONING (Diagnostic Audit Trail) ***")
        print(thought_summary.strip())

    print("=======================================================")

# --- EXECUTE ORCHESTRATION ---
ed_orchestrator("Process a new fever case and an urgent abdominal pain presentation.")

✅ Gemini client configured for **gemini-3-pro-preview**.

ED Supervisor receives new task: Process a new fever case and an urgent abdominal pain presentation.

--- 🟢 DELEGATING TO LOW-THINK AGENT (Triage/Fast Data) ---
Agent 1 Result (LOW THINK): ```json
{
  "esi_level": 3,
  "heart_rate": 110
}
```
--------------------------------------------------
--- 🔴 DELEGATING TO HIGH-THINK AGENT (Diagnosis/Deep Think Mode) ---
Agent 2 Result (HIGH THINK):
This clinical scenario presents a classic "vascular catastrophe" with an ethical complication. The presence of the "classic triad" (abdominal pain, hypotension, pulsatile mass) in a 72-year-old gives a pre-test probability of a Ruptured Abdominal Aortic Aneurysm (rAAA) approaching 100%.

Here is the optimal 3-step immediate management plan and the differential diagnosis analysis.

### Immediate Management Plan

**Step 1: Hemodynamic Stabilization via "Permissive Hypotension" & Access**
*   **Action:** Immediately establish two large-bore IVs (1